# Assignment 3: "Search your transcripts. You will know it to be true."

## © Cristian Danescu-Niculescu-Mizil 2020

## CS/INFO 4300 Language and Information

### Due by midnight on Wednesday February 12th


This is an **individual** assignment.

If you use any outside sources (e.g. research papers, StackOverflow) please list your sources.

In this assignment we will explore the tradeoffs of information retrieval systems by finding newspaper quotes from "Keeping Up With The Kardashians". In this part you will experiment with variations of edit distance. In the next assignment we will contrast this with other similarity measures.


**Guidelines**

All cells that contain the blocks that read `# YOUR CODE HERE` are editable and are to be completed to ensure you pass the test-cases. Make sure to write your code where indicated.

All cells that read `YOUR ANSWER HERE` are free-response cells that are editable and are to be completed.

You may use any number of notebook cells to explore the data and test out your functions, although you will only be graded on the solution itself.

You are unable to modify the read-only cells.

You should also use Markdown cells to explain your code and discuss your results when necessary.
Instructions can be found [here](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html).

All floating point values should be printed with **2 decimal places** precision. You can do so using the built-in round function.

**Grading**

For code-completion questions you will be graded on passing the public test cases we have included, as well as any hidden test cases that we have supplemented to ensure that your logic is correct.

For free-response questions you will be manually graded on the quality of your answer.

**Learning Objectives**
- Practicing our understanding of edit distance
- Implementing edit distance
- Building intuition about different cost schemas

In [1]:
import numpy as np
import math
from IPython.display import HTML

# Motivation

The tabloids have been going crazy over our stars! The press took some  quotes from the show, including:
       
 - *"It's like a bunch of people running around talking about nothing."*
 - *"Never say to a famous person that this possible endorsment would bring 'er to the spot light."*
 - *"Your yapping is making my head ache!"*
 - *"I'm going to Maryland, did I tell you?"*
 
We need to find out who said each of these, and in which episode. But since we're information scientists, that's not enough. We want to build an efficient search engine for retrieving where such quotes come from in the future.

What makes this difficult is that journalists often modify the quotes, so exact matching will not always work.

In [3]:
queries = [u"It's like a bunch of people running around talking about nothing.",
           u"Never say to a famous person that this possible endorsment would bring 'er to the spot light.",
           u"She just told me to get in the car and buckle my seatbelt.",
           u"I'm going to Maryland, did I tell you?"]

### Load the data

Load the transcripts provided in the `kardashian-transcripts.json` file.

In [4]:
import json
with open("kardashian-transcripts.json", "r") as f:
    transcripts = json.load(f)

#we will analyze only a subset of the transcripts to reduce computation time
transcripts = transcripts[470:500]

### Reorganize the data

For this assignment, we'll consider documents to be individual message lines. The provided transcripts are grouped differently. We reorganize the data as a list of messages, where the messages are dictionary structures as provided.

In [5]:
flat_msgs = [m for transcript in transcripts for m in transcript]

In [28]:
flat_msgs[1]

{'episode_title': 'Keeping Up With the Kardashians - Delivering Baby Mason',
 'speaker': 'CRANE',
 'text': 'Wow! Who knew?',
 'timestamp': '00:07:40',
 'toks': ['wow', '!', 'who', 'knew', '?'],
 'transcript_id': 'kardashians2/209686'}

# Searching the collection

### Question 1a: Verbatim Search (Code Completion)

The first and easiest thing we might try to do is to directly compare the newspaper quote to the transcript strings.  If the press just copy-pasted from the transcript website, this might work. We want to find all messages that include a given quote exactly.

To do this, write a function `verbatim_search` that looks for exact matches of a query in each message.

Hint: Use the `in` keyword. Ex. `'efg' in 'cdefgh'` evaluates to `True`.

Note: Because this is a verbatim search, this method should be case-sensitive.

In [15]:
def verbatim_search(query, msgs):
    """ Verbatim search
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
        
    msgs: list of dicts,
        Each message in this list has a 'text' field with
        the raw document.
    
    Returns
    =======
    result: list of messages
        All messages that exactly contain the query string.
    
    """
    # YOUR CODE HERE
    mess= []
    for epi in msgs:
        if query in epi['text']:
            mess.append(epi)
            
    return mess

In [16]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that tokenize returns the correct output"""
assert len(verbatim_search(queries[0], flat_msgs)) == 1
assert len(verbatim_search(queries[2], flat_msgs)) == 0

# This will nicely output the results
for no, query in enumerate(queries):
    print("[QUERY {}] : ".format(no), query)
    print("===")
    results = verbatim_search(query, flat_msgs)
    for msg in results:
        print("RESULT: {}: {}\n\t({})\n".format(
                                        msg['speaker'],
                                        msg['text'],
                                        msg['episode_title']))
    if not len(results):
        print("NO RESULTS")
    print()
    

[QUERY 0] :  It's like a bunch of people running around talking about nothing.
===
RESULT: BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)


[QUERY 1] :  Never say to a famous person that this possible endorsment would bring 'er to the spot light.
===
NO RESULTS

[QUERY 2] :  She just told me to get in the car and buckle my seatbelt.
===
NO RESULTS

[QUERY 3] :  I'm going to Maryland, did I tell you?
===
NO RESULTS




### Question 1b: Verbatim Search (Free Response)

Would you want to use a verbatim search for extracting the closest quotes from the transcripts? Why or why not?

YOUR ANSWER HERE

No, i won't. Because verbatim search is accurate search, meaning that we can only find a message iff the message is exactly the same as the query. In most of the cases, these query are kind of paraphase the message, we want a search engine that could match a message and query roughly.

### Question 2: Levenshtein Edit Distance (Code Completion)

In this section, we will now try using the more flexible edit distance metric (Levenshtein) to see if this improves the quality of our search results. For each query, we will need to loop over each message, and compute the edit distance each time -- like with `verbatim_search`, there are no obvious shortcuts.

We've provided you with code to compute the edit matrix that we discussed in class. You will need to complete the code stub below. (You can refer to [the worksheet we used in class](http://www.cs.cornell.edu/courses/cs4300/2020sp/Slides/Lecture2worksheet.pdf) if you need a refresher on how this works.)

Note: It is ok for there to be duplicates in your results.

In [41]:
def insertion_cost(message, j):
    return 1

def deletion_cost(query, i):
    return 1

def substitution_cost(query, message, i, j):
    if query[i-1] == message[j-1]:
        return 0
    else:
        return 1
    
curr_insertion_function = insertion_cost
curr_deletion_function = deletion_cost
curr_substitution_function = substitution_cost

def edit_matrix(query, message):
    """ calculates the edit matrix
    
    Arguments
    =========
    
    query: query string,
        
    message: message string,
    
    m: length of query + 1,
    
    n: length of message + 1,
    
    Returns:
        edit matrix {(i,j): int}
    """
    
    m = len(query) + 1
    n = len(message) + 1

    chart = {(0, 0): 0}
    for i in range(1, m): 
        chart[i,0] = chart[i-1, 0] + curr_deletion_function(query, i) 
    for j in range(1, n): 
        chart[0,j] = chart[0, j-1] + curr_insertion_function(message, j)
    for i in range(1, m):
        for j in range(1, n):
            chart[i, j] = min(
                chart[i-1, j] + curr_deletion_function(query, i),
                chart[i, j-1] + curr_insertion_function(message, j),
                chart[i-1, j-1] + curr_substitution_function(query, message, i, j)
            )
    return chart

def edit_distance(query, message):
    """ Edit distance calculator
    
    Arguments
    =========
    
    query: query string,
        
    message: message string,
    
    Returns:
        edit cost (int)
    """
        
    query = query.lower()
    message = message.lower()
    
    # YOUR CODE HERE
    chart = edit_matrix(query, message)
    return chart[len(query), len(message)]

def edit_distance_search(query, msgs):
    """ Edit distance search
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
        
    msgs: list of dicts,
        Each message in this list has a 'text' field with
        the raw document.
    
    Returns
    =======
    
    result: list of (score, message) tuples.
        The result list is sorted by score such that the closest match
        is the top result in the list.
    
    """
    # YOUR CODE HERE
    score_tuple = []
    for epi in msgs:
        edit_cost = edit_distance(query, epi['text'])
        score_tuple.append((edit_cost, epi))
    return sorted(score_tuple,key=lambda x: x[0])

When the edit_distance_search function is completed you can run the lines below to print out the best matches for each query string.

In [42]:
# This is an autograder test. Here we can test the function you just wrote above.
score, _ = edit_distance_search(queries[1], flat_msgs)[0]
assert score == 42


# note: this will take about 1-2 minutes to execute
top_10 = []
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    for score, msg in edit_distance_search(query, flat_msgs)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            msg['speaker'],
            msg['text'],
            msg['episode_title']))
        top_10.append(msg)
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[0.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[39.00] JONATHAN: It's like going to Justin Timberlake and talking about 'N Sync, like...
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[41.00] SIMON: I'll literally, at this point, take anything.
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[41.00] SHIEH: It's a complete approach to skin rejuvenation.
	(Keeping Up With the Kardashians - Leaving the Nest)
[41.00] KHLOE: I'm not going and you're not going.
	(Keeping Up With the Kardashians - Kim Becomes a Diva)
[41.00] KIM: I'm going to be riding around in my new Bentley.
	(Keeping Up With the Kardashians - Kim Becomes a Diva)
[41.00] BRUCE: The guy's down; I feel good about this w

# Question 2b: Query Discussion and Analysis (Free Response)

What do you notice about the search above?
1. Discuss why it worked, or why it might not have worked, for each query. 

2. Do you notice anything different about the costs to those discussed in lecture? 

3. Please fill out the table below with the costs from running the Levenshtein algorithm discussed in lecture and the cost from using the edit_distance function we provided above. Copy-and-paste this markdown table into the cell below for your answers:
```
| Operation    | Cost (Lecture)|  Cost (edit_distance)  | Example
| :----------: |:------------- | :-------------------------- | --------
| Addition     | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "aab"
| Deletion     | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "a"
| None         | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "aa"
| Substitution | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "ab"
| Multiple     | YOUR ANSWER   | YOUR ANSWER                 | "trawl" --> "claw"
```

YOUR ANSWER HERE

1. This might not have worked, since the query here is a sentence with a lot of words and spaces, if we used this method, we may ignore the sequence of the words in a sentence. For example, if a sentence_A is "Kardshina is a beauty", sentence_B is "beauty is Kardshina", and sentence_C is "Kardshian is a bread", in this way, sentence_A and sentence_c edit distance =6 will be lower than sentence_A and sentence_B=17, however, in fact sentence_A and B are more closer.

2. Yes, there are some differnce, for example, in the subsitute part, in the lecture, we think the cost is 2, because we delete and insert, but in this case, the substitute cost is 1.

3.

```

| Operation    | Cost (Lecture)|  Cost (edit_distance)       | Example
| :----------: |:------------- | :-------------------------- | --------
| Addition     | 1             | 1                           | "aa" -> "aab"
| Deletion     | 1             | 1                           | "aa" -> "a"
| None         | 0             | 0                           | "aa" -> "aa"
| Substitution | 2             | 1                           | "aa" -> "ab"
| Multiple     | 5             | 3                           | "trawl" --> "claw"
```

### Question 3: Implementing Edit Operations
The purpose of edit distance is not just for us to understand the lowest distance between two strings, but also what edits can be performed to turn one string into another. Using the edit matrix, it is possible to traverse backward and get a series of operations back that tell us how one string can be altered to produce another string.

We would like you to implement a function that will tell us how to write our message string as a function of our query string. We have provided you a skeleton code to help with this implementation.

Below is an example of the operations required to turn 'Kardashians' into 'Dalmatians'.

In [84]:
a = "kardashians"
b = "dalmatians"
edits = [('replace', 0, 0), ('replace', 2, 2), ('replace', 3, 3), ('delete', 5, 5), ('replace', 6, 5)]
print('edit operations:', edits)

edit operations: [('replace', 0, 0), ('replace', 2, 2), ('replace', 3, 3), ('delete', 5, 5), ('replace', 6, 5)]


In [80]:
def get_editops(str_a, str_b):
    
    """ Get Edit Operations
    
    Arguments
    =========
    
    str_a: the original query string.
                
    str_b: the message that we are comparing the query to.
    
    Returns
    =======
    
    An ordered minimum list of edit operations needed to turn
    str_a into str_b. Note that the number of edit operations
    returned should never exceed the total edit distance between
    str_a and str_b.
    
    These operations should be in a list of 3-tuples that take
    the form:
       * ('delete', x, y) for a deletion
       * ('insert', x, y) for an insertion
       * ('replace', x, y) for a replacement
    Where x and y are the positions of the characters the operation
    applies to. x is the position of the query string and y is
    the position of the message string.
    
    You should feel free to leverage any of the functions we
    have already provided you.
    
    Hint: To calculate these operations, you should start in 
    the bottom right of the matrix (with the winning edit distance) 
    and traverse up the optimal possible path to the top left
    corner of the matrix.
    
    Hint: Remember that insertions and deletions always have a
    cost, and substitutions sometimes have a cost.
    
    Note: Don't forget to use the cost functions we've provided
    instead of relying on hard coding these values!

    """
    
    # initialise variables
    i = len(str_a)
    j = len(str_b)
    str_a = str_a.lower()
    str_b = str_b.lower()
    matrix = edit_matrix(str_a, str_b)
    edit_ops = []
    
    # this algorithm ends once we have worked our way
    # back to the (0,0) location in the matrix
    
    while (i,j) != (0,0):
        curr_cell = matrix[i, j]
        
        # what must you do if you can no longer traverse left?
        if (i != 0 and j == 0):
            # YOUR CODE HERE
            edit_ops.append(('delete', i-1,0))
            i-=1
            
        # what must you do if you can no longer traverse up?
        elif (i == 0 and j != 0):
            # YOUR CODE HERE
            edit_ops.append(('insert', 0, j-1))
            j-=1
            
        # this will get called if you can traverse either left,
        # up, or diagonally
        else:
            up_cell = matrix[i-1, j]
            left_cell = matrix[i, j-1]
            diag_cell = matrix[i-1, j-1]
            
            # we calculate the minimum as before
            curr_min = min(
                up_cell + curr_deletion_function(str_a, i),
                left_cell + curr_insertion_function(str_b, j), 
                diag_cell + curr_substitution_function(str_a, str_b, i, j)
            )
            
            # for the shortest path, traverse up towards the
            # left corner in any possible direction that
            # minimizes the total cost
            
            # YOUR CODE HERE
            if curr_min == diag_cell:
                i-=1
                j-=1
                
                
            elif (curr_min==up_cell+1):
                edit_ops.append(('delete', i-1,j-1))
                i = i-1
                
            elif (curr_min==left_cell+1):
                edit_ops.append(('insert', i-1,j-1))
                j = j-1
                
            elif (curr_min==diag_cell+1):
                edit_ops.append(('replace', i-1,j-1))
                i = i-1
                j = j-1
            
    #this will return a reversed list of edit operations
    return edit_ops[::-1]

In [81]:
# This is an autograder test. Here we can test the function you just wrote above.
a = "kardashians"
b = "dalmatians"
edits = get_editops(a,b)
assert edits == ([('replace', 0, 0), ('replace', 2, 2), ('replace', 3, 3), ('delete', 5, 5), ('replace', 6, 5)])\
    or ([('replace', 0, 0), ('replace', 2, 2), ('replace', 3, 3), ('replace', 5, 5), ('delete', 6, 6)])
assert (len(edits)) == edit_distance(a,b)


### Question 4a: Visualizing Edit Distance (Code Completition)

We've provided some code below that displays the edits that need to be made to a string to transform it into another string. Your job in this assignment is visualize the edits for the 4 queries at the top of this assignment to get a feel for how edit distance is working. 

For each of your searches in Q2, we would now like you to display the edit transformations from each query to each of its top-10 search results.

In [86]:
def print_edits(str_a, str_b, edits):
    output = [[char] for char in str_a]
    indices = np.arange(len(str_a) + 1)
    for op, src, dest in edits:
        if op == 'insert':
            src = indices[src]
            output.insert(src, ["<span class='add'>{}</span>".format(str_b[dest])])
            indices += 1
        elif op == 'replace':
            src = indices[src]
            src_char = output[src][0]
            output[src] = output[src][1:]
            output[src].append("<span class='del'>{}</span><span class='add'>{}</span>".format(src_char, str_b[dest]))
        elif op == 'delete':
            src = indices[src]
            src_char = output[src].pop()
            output[src].append("<span class='del'>{}</span>".format(src_char))
    
    return "<div class='edit'>{}</div>".format("".join("".join(stack) for stack in output))

HTML("""
<style type="text/css">

.edit {font-size: 20px;}
.del {text-decoration: line-through; color: #aaa;}
.add {color: green; font-weight: bold;}
</style>
""")

We can now vizualise the edit distances and operations we were working with before!

In [87]:
a = "kardashians"
b = "dalmatians"
edits = [('replace', 0, 0), ('replace', 2, 2), ('replace', 3, 3), ('delete', 5, 5), ('replace', 6, 5)]

display(HTML(print_edits(a, b, edits)))

#### Use the provided `HTML` and `print_edits` functions to visualize the edits for each of the search results you computed in Q2. You should display the top-10 scoring search results for each query.

Hint: To display edits from the `HTML`, you will need to use the `display` method as shown above.

Hint: You will want to use `top_10` here.

In [91]:
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))
    
    # YOUR CODE HERE
    scores = edit_distance_search(query, flat_msgs)
    for mess in scores[:10]:
        message = mess[1]['text']
        edits= get_editops(query, message)
        display(HTML(print_edits(query, message, edits)))

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################


#############################################################################################
Never say to a famous person that this possible endorsment would bring 'er to the spot light.
#############################################################################################


##########################################################
She just told me to get in the car and buckle my seatbelt.
##########################################################


######################################
I'm going to Maryland, did I tell you?
######################################


### Question 4b: Visualizing Edit Distance (Free Response)
What do you observe from the visualized edits above? 

Would you want to use this method in a search engine? Why or why not?

Include a short discussion of what seems to work well and what doesn't seem to work well and why this might be.

YOUR ANSWER HERE

1. In most of the cases, the original sentence(query) is deleted or inserted in most of the cases, while only a few of the characters are left, this means that there are too much changes to do in order to translate one sentence into another.

2. just like what i said in last problem, This might not have worked, since the query here is a sentence with a lot of words and spaces, if we used this method, we may ignore the sequence of the words in a sentence. For example, if a sentence_A is "Kardshina is a beauty", sentence_B is "beauty is Kardshina", and sentence_C is "Kardshian is a bread", in this way, sentence_A and sentence_c edit distance =6 will be lower than sentence_A and sentence_B=17, however, in fact sentence_A and B are more closer.

### Question 5a: Changing the costs (Code Completion)

Thus far all our code has been using standard costs. Sometimes we might want to alter the edit-distance penalty depending on the type of change we are making. In practice, the use of a simple uniform cost for all changes is suboptimal, since certain changes are more likely to happen than others. For instance, it is easier to mix up characters that are adjacent to each other on the keyboard (typos), and we'd like you to **reduce the cost of such substitutions of adjacent keyboard letters to 1.75 instead of 2**.

In [93]:
# we provide you with a list of adjacent characters
adj_chars = [('a', 'q'), ('a', 's'), ('a', 'z'), ('b', 'g'), ('b', 'm'), ('b', 'n'), ('b', 'v'), ('c', 'd'),
             ('c', 'v'), ('c', 'x'), ('d', 'c'), ('d', 'e'), ('d', 'f'), ('d', 's'), ('e', 'd'), ('e', 'r'),
             ('e', 'w'), ('f', 'd'), ('f', 'g'), ('f', 'r'), ('f', 'v'), ('g', 'b'), ('g', 'f'), ('g', 'h'),
             ('g', 't'), ('h', 'g'), ('h', 'j'), ('h', 'm'), ('h', 'n'), ('h', 'y'), ('i', 'k'), ('i', 'o'),
             ('i', 'u'), ('j', 'h'), ('j', 'k'), ('j', 'u'), ('k', 'i'), ('k', 'j'), ('k', 'l'), ('l', 'k'),
             ('l', 'o'), ('m', 'b'), ('m', 'h'), ('n', 'b'), ('n', 'h'), ('o', 'i'), ('o', 'l'), ('o', 'p'),
             ('p', 'o'), ('q', 'a'), ('q', 'w'), ('r', 'e'), ('r', 'f'), ('r', 't'), ('s', 'a'), ('s', 'd'),
             ('s', 'w'), ('s', 'x'), ('t', 'g'), ('t', 'r'), ('t', 'y'), ('u', 'i'), ('u', 'j'), ('u', 'y'), 
             ('v', 'b'), ('v', 'c'), ('v', 'f'), ('w', 'e'), ('w', 'q'), ('w', 's'), ('x', 'c'), ('x', 's'), 
             ('x', 'z'), ('y', 'h'), ('y', 't'), ('y', 'u'), ('z', 'a'), ('z', 'x')]

In [96]:
def substitution_cost_adj(query, message, i, j):
    """
    Custom substitution cost:
    The cost is 1.75 when substituting a pair of characters that can be found in adj_chars
    Otherwise, the cost is 2. (Not 1 as it was before!)
    """
    # YOUR CODE HERE
    if query[i-1] == message[j-1]:
        return 0
    elif (query[i-1],message[j-1]) in adj_chars:
        return 1.75
    else:
        return 2
    
curr_substitution_function = substitution_cost_adj

In [97]:
assert edit_distance("Levenshtein","Levenstein") == 1
assert edit_distance("Levenshtein","Lebenshtein") == 1.75

Run the following code below to reorder the top ten closest matches for each query using our new custom edit-distance discounts.

Note: It is ok to obtain duplicate results.

In [101]:
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    # we compare the query with top_10 to save time because
    # we already know that top_10 will be close to the query
    # so there is no need to check all flat_msgs
    for score, msg in edit_distance_search(query, top_10)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            msg['speaker'],
            msg['text'],
            msg['episode_title']))
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[0.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[49.25] KHLOE: I'm not going and you're not going.
	(Keeping Up With the Kardashians - Kim Becomes a Diva)
[50.25] BRUCE: I'm looking forward to this.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[50.75] JONATHAN: I've literally been reading about...
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[52.50] KOURTNEY: I'm not going to go tell him anything.
	(Keeping Up With the Kardashians - Blame It on the Alcohol)
[53.50] SIMON: I'll literally, at this point, take anything.
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[53.50] KIM: I'm going to be riding around in my new Bentley.
	(Keeping Up With the Kardashians - Kim Bec

### Question 5b: Changing the costs (Free Response)

How do these results compare to that of default `edit_distance` metric?

YOUR ANSWER HERE

1. In algorithm, this one cost more in substituting the characters, while the default one the insertion, deletion and substitution cost the same.

SO, in this case, the top 10 are composed of the sentences with more insertion and deletion, for example, a lot of short sentences are chosen into the top 10 cloest sentences.

### Question 5c: Changing the costs (Code Completition)

We'd now like you to modify your `custom_edit_distance` costs once more such that substitutions for adjacent keyboard letters are still 1.75, but substitutions for non-adjacent keyboard letters are penalized with a much higher cost of 4.

Complete the code stub below and run the cells below to recompute our queries once more.

In [102]:
def substitution_cost_non_adj(query, message, i, j):
    """
    Q4c - Custom substitution cost (part 2):
    The cost is 1.75 when substituting a pair of characters that can be found in adj_chars
    Otherwise, the cost is *** 4 ***. (Not 1 as it was before!)
    """
    
    # YOUR CODE HERE
    if query[i-1] == message[j-1]:
        return 0
    elif (query[i-1],message[j-1]) in adj_chars:
        return 4
    else:
        return 2
    
curr_substitution_function = substitution_cost_non_adj

In [103]:
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    for score, msg in edit_distance_search(query, top_10)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            msg['speaker'],
            msg['text'],
            msg['episode_title']))
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[0.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[50.00] KHLOE: I'm not going and you're not going.
	(Keeping Up With the Kardashians - Kim Becomes a Diva)
[51.00] JONATHAN: I've literally been reading about...
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[51.00] BRUCE: I'm looking forward to this.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[53.00] KOURTNEY: I'm not going to go tell him anything.
	(Keeping Up With the Kardashians - Blame It on the Alcohol)
[54.00] SIMON: I'll literally, at this point, take anything.
	(Keeping Up With the Kardashians - Delivering Baby Mason)
[55.00] SHIEH: It's a complete approach to skin rejuvenation.
	(Keeping Up With the Kardashians - Leaving

### Question 5d: Changing the costs (Free Response)
What do you notice about the results of this search as compared that of our first `custom_edit_distance` and why do we see this?

YOUR ANSWER HERE

Compare to other lower substitution cost, this way the substitution cost is high especially of this closet chars. In most of the cases, this kind of close char substitution is not rank as the top 10 cloest anymore. Instead some short sentences are chosen as the top 10.
